In [1]:
import requests
import pandas as pd
from datetime import date
import sqlalchemy as db

In [2]:
today = date.today()
dateval = today.strftime("%m/%d/%Y")
min_rank = 1
max_rnk = 100
total = 1000
responses = []

# url = f"https://clinicaltrials.gov/api/query/full_studies?expr=COVID-19+AND+SEARCH%5BLocation%5D%28AREA%5BLocationCountry%5D+United+States%29&min_rnk={min_rank}&max_rnk={max_rnk}&fmt=json"
# response = requests.get(url)
# json = response.json()

try:
    while max_rnk < total:
        url = f"https://clinicaltrials.gov/api/query/full_studies?expr=COVID-19+AND+SEARCH%5BLocation%5D%28AREA%5BLocationCountry%5D+United+States%29&min_rnk={min_rank}&max_rnk={max_rnk}&fmt=json"
        response = requests.get(url)
        json = response.json()
        for rank in json['FullStudiesResponse']['FullStudies']:
            responses.append(rank)
        min_rank += 100
        max_rnk += 100
except:
    pass




In [3]:
len(responses)

138

In [4]:
responses[0]

{'Rank': 1,
 'Study': {'ProtocolSection': {'IdentificationModule': {'NCTId': 'NCT04349202',
    'OrgStudyIdInfo': {'OrgStudyId': '2020-134'},
    'Organization': {'OrgFullName': 'William Beaumont Hospitals',
     'OrgClass': 'OTHER'},
    'BriefTitle': 'Beaumont Health Large-scale Automated Serologic Testing for COVID-19',
    'OfficialTitle': 'Beaumont Health Large-scale Automated Serologic Testing for COVID-19',
    'Acronym': 'BLAST COVID-19'},
   'StatusModule': {'StatusVerifiedDate': 'April 2020',
    'OverallStatus': 'Recruiting',
    'ExpandedAccessInfo': {'HasExpandedAccess': 'No'},
    'StartDateStruct': {'StartDate': 'April 2020',
     'StartDateType': 'Anticipated'},
    'PrimaryCompletionDateStruct': {'PrimaryCompletionDate': 'June 2021',
     'PrimaryCompletionDateType': 'Anticipated'},
    'CompletionDateStruct': {'CompletionDate': 'June 2021',
     'CompletionDateType': 'Anticipated'},
    'StudyFirstSubmitDate': 'April 13, 2020',
    'StudyFirstSubmitQCDate': 'April 13,

In [5]:
n = 0
main_list = []
intervention_list = []
phase_list = []

for rank in responses:
    try:
        nctid = responses[n]["Study"]["ProtocolSection"]["IdentificationModule"]["NCTId"]
        brieftitle = responses[n]["Study"]["ProtocolSection"]["IdentificationModule"]["BriefTitle"]
        start_date = responses[n]["Study"]["ProtocolSection"]["StatusModule"]["StartDateStruct"]["StartDate"]
        completiondate = responses[n]["Study"]["ProtocolSection"]["StatusModule"]["CompletionDateStruct"]["CompletionDate"]
        overallstatus = responses[n]["Study"]["ProtocolSection"]["StatusModule"]["OverallStatus"]
        leadsponsor = responses[n]["Study"]["ProtocolSection"]["SponsorCollaboratorsModule"]["LeadSponsor"]["LeadSponsorName"]
        contactname = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["CentralContactList"]["CentralContact"][0]["CentralContactName"]
        contactphone = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["CentralContactList"]["CentralContact"][0]["CentralContactPhone"]
        contactemail = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["CentralContactList"]["CentralContact"][0]["CentralContactEMail"]
        briefsummary = responses[n]["Study"]["ProtocolSection"]["DescriptionModule"]["BriefSummary"]
        enrollmentcount = responses[n]["Study"]["ProtocolSection"]["DesignModule"]["EnrollmentInfo"]["EnrollmentCount"]
        primaryoutcomedesc = responses[n]["Study"]["ProtocolSection"]["OutcomesModule"]["PrimaryOutcomeList"]["PrimaryOutcome"][0]["PrimaryOutcomeMeasure"]
        facility = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["LocationList"]["Location"][0]["LocationFacility"]
        locationcity = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["LocationList"]["Location"][0]["LocationCity"]
        locationstate = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["LocationList"]["Location"][0]["LocationState"]
        locationzip = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["LocationList"]["Location"][0]["LocationZip"]
        locationcountry = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["LocationList"]["Location"][0]["LocationCountry"]
        main_dict = {"nctid":nctid,
               "brieftitle":brieftitle,
               "start_date":start_date,
               "completiondate":completiondate,
               "overallstatus":overallstatus,
               "leadsponsor":leadsponsor,
               "contactname":contactname,
               "contactphone": contactphone,
               "contactemail":contactemail,
               "briefsummary":briefsummary,
               "enrollmentcount":enrollmentcount,
               "primaryoutcomedesc":primaryoutcomedesc,
               "facility":facility,
               "locationcity":locationcity,
               "locationstate":locationstate,
               "locationzip":locationzip,
               "locationcountry":locationcountry,
               "statusDate":dateval}
        main_list.append(main_dict)
        for intervention in responses[n]["Study"]["ProtocolSection"]["ArmsInterventionsModule"]["InterventionList"]["Intervention"]:
            interventionname = intervention["InterventionName"]
            interventiontype = intervention["InterventionType"]
            interventiondesc = intervention["InterventionDescription"]
            intervention_dict = {
                "nctid": nctid,
                "interventionname": interventionname,
                "interventiontype": interventiontype,
                "interventiondesc": interventiondesc,
                "statusDate": dateval}
            intervention_list.append(intervention_dict)
        if hasattr(responses[n]["Study"]["ProtocolSection"]["DesignModule"], "PhaseList"):
            for phase in responses[n]["Study"]["ProtocolSection"]["DesignModule"]["PhaseList"]["Phase"]:
                phase_dict = {
                    "nctid": nctid,
                    "phase": phase,
                    "statusDate": dateval
                }
                phase_list.append(phase_dict)         
        n+=1
    except:
        n+=1
        pass


In [6]:
trials_found = len(main_list)
trials_found

111

In [7]:
covid_df = pd.DataFrame(main_list)
intervention_df = pd.DataFrame(intervention_list)
phase_df = pd.DataFrame(phase_list)

In [8]:
covid_df.head()

,nctid,brieftitle,start_date,completiondate,overallstatus,leadsponsor,contactname,contactphone,contactemail,briefsummary,enrollmentcount,primaryoutcomedesc,facility,locationcity,locationstate,locationzip,locationcountry,statusDate
0,NCT04349202,Beaumont Health Large-scale Automated Serologi...,April 2020,June 2021,Recruiting,William Beaumont Hospitals,"Maureen Cooney, RN, BSN",248-551-0099,Maureen.Cooney@beaumont.org,The purpose of this study is to determine how ...,50000,Prevalence COVID antibodies in employees of Be...,Beaumont Health System,Royal Oak,Michigan,48073,United States,04/19/2020
1,NCT04329832,Hydroxychloroquine vs. Azithromycin for Hospit...,"March 30, 2020","December 31, 2021",Recruiting,"Intermountain Health Care, Inc.","Valerie T Aston, MBA",8015074606,Valerie.Aston@imail.org,This study will compare two drugs (hydroxychlo...,300,COVID Ordinal Outcomes Scale at 14 days,Intermountain Medical Center,Murray,Utah,84107,United States,04/19/2020
2,NCT04334382,Hydroxychloroquine vs. Azithromycin for Outpat...,"April 2, 2020","December 31, 2021",Recruiting,"Intermountain Health Care, Inc.","Valerie T Aston, MBA",8015074606,Valerie.Aston@imail.org,This study will compare two drugs (hydroxychlo...,1550,Hospitalization within 14 days of enrollment,Intermountain Medical Center,Murray,Utah,84107,United States,04/19/2020
3,NCT04331366,Bidirectional Oxygenation Valve in the Managem...,"April 8, 2020",May 2020,Recruiting,Emory University,"Jeffrey Miller, MD",404-778-7200,jeffrey.miller@emory.edu,The objective of this study is to determine th...,5,Change in Oxygen Saturation by Pulse Oximetry,Emory University Hospital,Atlanta,Georgia,30322,United States,04/19/2020
4,NCT04339998,Assessment of Exam Findings in Coronavirus Dis...,"April 15, 2020",October 2020,Not yet recruiting,University of Minnesota,"Matthew Yocum, MD",612-626-8015,yocum007@umn.edu,Specific Aims:\n\nThe investigators will prosp...,500,POCUS Score - Lungs,University of Minnesota Medical Center (UMMC),Minneapolis,Minnesota,55455,United States,04/19/2020


In [9]:
covid_df.describe()

,nctid,brieftitle,start_date,completiondate,overallstatus,leadsponsor,contactname,contactphone,contactemail,briefsummary,enrollmentcount,primaryoutcomedesc,facility,locationcity,locationstate,locationzip,locationcountry,statusDate
count,111,111,111,111,111,111,111,111,111,111,111,111,111,111,111,111,111,111
unique,111,111,44,65,2,73,105,104,103,111,66,108,92,51,27,73,1,1
top,NCT04343898,COVID-19 Remote Monitoring,April 2020,"December 31, 2021",Recruiting,Duke University,"Sabine Hazan, MD",541-484-2123,estice@ori.org,Minimal risk research study:\n\nComparing poly...,200,Mortality,Massachusetts General Hospital,New York,New York,10032,United States,04/19/2020
freq,1,1,23,7,74,6,2,2,2,1,5,3,4,10,16,4,111,111


In [10]:
#check if there are Null cells in covid_df
covid_df.isnull().values.any()

False

In [11]:
# look at datatypes
# covid_df.dtypes

In [12]:
# remove day from start date column
covid_df['starting_date']=covid_df['start_date'].str.split(r'\s+').apply(lambda x: x[0:1] + x[2:] if len(x) > 2 else x).str.join(' ')
# covid_df
covid_df['completion_date']=covid_df['completiondate'].str.split(r'\s+').apply(lambda x: x[0:1] + x[2:] if len(x) > 2 else x).str.join(' ')

# drop initial date columns
covid_df = covid_df.drop(['start_date','completiondate' ], axis=1)
covid_df


,nctid,brieftitle,overallstatus,leadsponsor,contactname,contactphone,contactemail,briefsummary,enrollmentcount,primaryoutcomedesc,facility,locationcity,locationstate,locationzip,locationcountry,statusDate,starting_date,completion_date
0,NCT04349202,Beaumont Health Large-scale Automated Serologi...,Recruiting,William Beaumont Hospitals,"Maureen Cooney, RN, BSN",248-551-0099,Maureen.Cooney@beaumont.org,The purpose of this study is to determine how ...,50000,Prevalence COVID antibodies in employees of Be...,Beaumont Health System,Royal Oak,Michigan,48073,United States,04/19/2020,April 2020,June 2021
1,NCT04329832,Hydroxychloroquine vs. Azithromycin for Hospit...,Recruiting,"Intermountain Health Care, Inc.","Valerie T Aston, MBA",8015074606,Valerie.Aston@imail.org,This study will compare two drugs (hydroxychlo...,300,COVID Ordinal Outcomes Scale at 14 days,Intermountain Medical Center,Murray,Utah,84107,United States,04/19/2020,March 2020,December 2021
2,NCT04334382,Hydroxychloroquine vs. Azithromycin for Outpat...,Recruiting,"Intermountain Health Care, Inc.","Valerie T Aston, MBA",8015074606,Valerie.Aston@imail.org,This study will compare two drugs (hydroxychlo...,1550,Hospitalization within 14 days of enrollment,Intermountain Medical Center,Murray,Utah,84107,United States,04/19/2020,April 2020,December 2021
3,NCT04331366,Bidirectional Oxygenation Valve in the Managem...,Recruiting,Emory University,"Jeffrey Miller, MD",404-778-7200,jeffrey.miller@emory.edu,The objective of this study is to determine th...,5,Change in Oxygen Saturation by Pulse Oximetry,Emory University Hospital,Atlanta,Georgia,30322,United States,04/19/2020,April 2020,May 2020
4,NCT04339998,Assessment of Exam Findings in Coronavirus Dis...,Not yet recruiting,University of Minnesota,"Matthew Yocum, MD",612-626-8015,yocum007@umn.edu,Specific Aims:\n\nThe investigators will prosp...,500,POCUS Score - Lungs,University of Minnesota Medical Center (UMMC),Minneapolis,Minnesota,55455,United States,04/19/2020,April 2020,October 2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,NCT03710746,Project Health: Enhancing Effectiveness of a D...,Recruiting,Oregon Research Institute,"Eric M Stice, Ph.D.",541-484-2123,estice@ori.org,This project seeks to improve the effectivenes...,450,Body Fat,Oregon Research Institute,Eugene,Oregon,97403,United States,04/19/2020,October 2018,July 2023
107,NCT04173663,Advocating for Supports to Improve Service Tra...,Recruiting,Vanderbilt University Medical Center,"Julie Lounds Taylor, PhD",615-343-5659,julie.l.taylor@vumc.org,This is a randomized intervention study to tes...,180,Change in Parental Knowledge about adult servi...,University of Illinois at Urbana-Champaign,Chicago,Illinois,60007,United States,04/19/2020,December 2019,May 2024
108,NCT04333732,CROWN CORONATION: Chloroquine RepurpOsing to h...,Not yet recruiting,Washington University School of Medicine,"Linda Yun, BS",314-273-2240,lindayun@wustl.edu,Healthcare workers are at the frontline of the...,55000,Symptomatic COVID-19,Washington University School of Medicine,Saint Louis,Missouri,63110,United States,04/19/2020,April 2020,February 2021
109,NCT02915198,Investigation of Metformin in Pre-Diabetes on ...,Recruiting,VA Office of Research and Development,"Gregory G Schwartz, PhD MD",(720) 723-6070,Gregory.Schwartz@va.gov,This research will help us to learn if the med...,7868,"Time in days to death, non-fatal myocardial in...","Phoenix VA Health Care System, Phoenix, AZ",Phoenix,Arizona,85012,United States,04/19/2020,February 2019,August 2024


In [13]:
engine = db.create_engine('sqlite:///COVID-Clinical-Trials.sqlite')

metadata = db.MetaData()

covid_df.to_sql("COVID_ClinicalTrials", con=engine, if_exists="replace")
intervention_df.to_sql("Interventions", con=engine, if_exists="replace")
phase_df.to_sql("Phase_Recordings", con=engine, if_exists="replace")

In [14]:
connection = engine.connect()
ziplist = connection.execute("SELECT DISTINCT locationzip FROM COVID_ClinicalTrials")


In [15]:
ziplist

In [16]:
d, a = {}, []
for rowproxy in ziplist:
    # rowproxy.items() returns an array like [(key0, value0), (key1, value1)] > thank you stackoverflow!
    for column, value in rowproxy.items():
        # build up the dictionary
        d = {**d, **{column: value}}
    a.append(d)
connection.close()
a

[{'locationzip': '48073'},
 {'locationzip': '84107'},
 {'locationzip': '30322'},
 {'locationzip': '55455'},
 {'locationzip': '62220'},
 {'locationzip': '27710'},
 {'locationzip': '85254'},
 {'locationzip': '07601'},
 {'locationzip': '96813'},
 {'locationzip': '94022'},
 {'locationzip': '60452'},
 {'locationzip': '93003'},
 {'locationzip': '19107'},
 {'locationzip': '60612'},
 {'locationzip': '66160'},
 {'locationzip': '60637'},
 {'locationzip': '98195'},
 {'locationzip': '80260'},
 {'locationzip': '20301'},
 {'locationzip': '80045'},
 {'locationzip': '10032'},
 {'locationzip': '94305'},
 {'locationzip': '19104'},
 {'locationzip': '10029'},
 {'locationzip': '64111'},
 {'locationzip': '21201'},
 {'locationzip': '94143'},
 {'locationzip': '02114-2621'},
 {'locationzip': '37232'},
 {'locationzip': '70121'},
 {'locationzip': '20892'},
 {'locationzip': '02114'},
 {'locationzip': '10016'},
 {'locationzip': '10467'},
 {'locationzip': '27157'},
 {'locationzip': '70112'},
 {'locationzip': '45267

In [17]:
a[0]["locationzip"]
ziplist = []
for n in a:
    ziplist.append(n["locationzip"])
    
ziplist

['48073',
 '84107',
 '30322',
 '55455',
 '62220',
 '27710',
 '85254',
 '07601',
 '96813',
 '94022',
 '60452',
 '93003',
 '19107',
 '60612',
 '66160',
 '60637',
 '98195',
 '80260',
 '20301',
 '80045',
 '10032',
 '94305',
 '19104',
 '10029',
 '64111',
 '21201',
 '94143',
 '02114-2621',
 '37232',
 '70121',
 '20892',
 '02114',
 '10016',
 '10467',
 '27157',
 '70112',
 '45267',
 '98133',
 '48202',
 '11501',
 '35222',
 '21287',
 '06105',
 '13326',
 '23249',
 '84108',
 '33136',
 '77030',
 '02115',
 '35233',
 '84132',
 '27701',
 '08901',
 '27713',
 '64114',
 '55415',
 '10010',
 '32224',
 '35294',
 '30030-1705',
 '63110',
 '61637',
 '27704',
 '91942',
 '11507',
 '90095',
 '92806',
 '92093',
 '85721',
 '91010',
 '97403',
 '60007',
 '85012']

In [18]:
from info import KEY
url = "http://open.mapquestapi.com/geocoding/v1/address?"
results = []
for n in ziplist:
    query = f"{url}key={KEY}&location={n}"
    response = requests.get(query)
    json = response.json()
    resultDict = {}
    latitude = json["results"][0]["locations"][0]["latLng"]["lat"]
    longitude = json["results"][0]["locations"][0]["latLng"]["lng"]
    locationzip = json["results"][0]["providedLocation"]["location"]
    resultDict = {
        "locationzip":locationzip,
        "longitude":longitude,
        "latitude":latitude
    }
    results.append(resultDict)
results

[{'locationzip': '48073', 'longitude': -83.16537, 'latitude': 42.510516},
 {'locationzip': '84107', 'longitude': 16.986129, 'latitude': 48.22045},
 {'locationzip': '30322', 'longitude': -84.324982, 'latitude': 33.79455},
 {'locationzip': '55455', 'longitude': -93.233774, 'latitude': 44.97545},
 {'locationzip': '62220', 'longitude': 36.148353, 'latitude': 50.419633},
 {'locationzip': '27710', 'longitude': 1.363306, 'latitude': 48.791159},
 {'locationzip': '85254', 'longitude': -111.956109, 'latitude': 33.619789},
 {'locationzip': '07601', 'longitude': -74.046952, 'latitude': 40.885173},
 {'locationzip': '96813', 'longitude': -157.856563, 'latitude': 21.305802},
 {'locationzip': '94022', 'longitude': -122.104306, 'latitude': 37.392784},
 {'locationzip': '60452', 'longitude': -87.753817, 'latitude': 41.606904},
 {'locationzip': '93003', 'longitude': -5.354353, 'latitude': 35.572052},
 {'locationzip': '19107', 'longitude': -75.158446, 'latitude': 39.951818},
 {'locationzip': '60612', 'long

In [19]:
results_df = pd.DataFrame(results)
results_df.head()

,locationzip,longitude,latitude
0,48073,-83.165370,42.510516
1,84107,16.986129,48.220450
2,30322,-84.324982,33.794550
3,55455,-93.233774,44.975450
4,62220,36.148353,50.419633


In [20]:
engine = db.create_engine('sqlite:///COVID-Clinical-Trials.sqlite')

metadata = db.MetaData()

results_df.to_sql("lat_long", con=engine, if_exists="replace")